In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from skimpy import skim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pdb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
np.random.seed(42)
# import pandas_profiling

In [ ]:
# subdirs = os.listdir('/Users/awxlong/Desktop/my-studies/temp_data/CRC/TCGA-CRC/gdc_download_20240629_132916.799980/')
cohort_name = 'COAD'
task_name = 'g0_arrest'
# Parent directory containing subdirectories with XML files
parent_directory = '/Users/awxlong/Desktop/my-studies/temp_data/CRC/TCGA-CRC/gdc_download_20240629_132916.799980/'
root_dir = '/Users/awxlong/Desktop/my-studies/hpc_exps/Data/'

df = pd.read_csv(f'{root_dir}{cohort_name}_WSI_clinical_features.csv')


In [ ]:
print(df.shape)
df.head()

In [ ]:
df['Gender'].value_counts()

In [ ]:
skim(df)

In [ ]:
# # Numerical features
# num_feats=[col for col in df.columns if df[col].dtypes != 'object']

# # Plot distribution of numerical columns
# fig=plt.figure(figsize=(20,30))
# for i, col in enumerate(num_feats):
#     plt.subplot(len(num_feats),1,1*i+1)
#     sns.distplot(df[col])
    
# fig.tight_layout()
# plt.show()

def icd10_to_description(icd10_codes):
    icd10_descriptions = {
        'C18.0': 'Malignant neoplasm of cecum',
        'C18.2': 'Malignant neoplasm of ascending colon',
        'C18.3': 'Malignant neoplasm of hepatic flexure',
        'C18.4': 'Malignant neoplasm of transverse colon',
        'C18.5': 'Malignant neoplasm of splenic flexure',
        'C18.6': 'Malignant neoplasm of descending colon',
        'C18.7': 'Malignant neoplasm of sigmoid colon',
        'C18.9': 'Malignant neoplasm of colon, unspecified',
        'C19.9': 'Malignant neoplasm of rectosigmoid junction'
    }
    
    descriptions = []
    for code in icd10_codes:
        if code in icd10_descriptions:
            descriptions.append(icd10_descriptions[code])
        else:
            descriptions.append('Unknown ICD-10 code')
    
    return descriptions


In [ ]:
print(df.columns)
print(df['Icd 10'].unique())
# print(df['Icd O 3 Site'].unique())

In [ ]:
df['Icd 10'].value_counts(normalize=True).plot(kind='bar')


### mode impute missing values

In [ ]:
def imputation(split, train_split=df):
   mode_values = train_split.mode().iloc[0]

   imputed_split = split.fillna(mode_values)

   return imputed_split



print("before imputation:", sum(df.isna().sum()))
# print("Number of NaN values per feature: ")
# df_train_na = df_train.apply(mode_imputation, axis = 1)
df = imputation(df)
print("after imputation:", sum(df.isna().sum()))

### convert from object to category

In [ ]:
categorical_vars = [col for col in df.columns if df[col].dtypes == 'object' and col != 'PatientID']
def cast_category(split, train_cat_vars=categorical_vars):
    split[train_cat_vars] = split[train_cat_vars].astype('category')
    # split = pd.get_dummies(split, columns=train_cat_vars, prefix= train_cat_vars)
    return split
# df_train[categorical_vars] = df_train[categorical_vars].astype('category')
df = cast_category(df)

In [ ]:
df.dtypes

### one hot encode categorical variables

In [ ]:
def one_hot_encode(split):
  cats = [col for col in split.columns if split[col].dtypes == 'category' and col != 'readmitted']
  split = pd.get_dummies(split, columns=cats, prefix= cats)
  return split

df = one_hot_encode(df)

In [ ]:
# df['Circumferential Resection Margin'].unique()


In [ ]:
print(df.shape)
df.head()